In [24]:
import pandas as pd
import numpy as np

## Dataset

In [25]:
sales = pd.read_csv("100_Sales.csv")
sales.dropna(axis=1,inplace=True)
sales.head(10)

,Region,Country,Item_Type,Sales_Channel,Order_Priority,Ship_Date,Unit_Cost,Total_Revenue,Total_Profit
0,Australia and Oceania,Tuvalu,Baby Food,Offline,H,27/06/2010,159.42,2533654.00,951410.50
1,Central America and the Caribbean,Grenada,Cereal,Online,C,15/09/2012,117.11,576782.80,248406.36
2,Europe,Russia,Office Supplies,Offline,L,05/08/2014,524.96,1158502.59,224598.75
3,Sub_Saharan Africa,Sao Tome and Principe,Fruits,Online,C,07/05/2014,6.92,75591.66,19525.82
4,Sub_Saharan Africa,Rwanda,Office Supplies,Offline,L,02/06/2013,524.96,3296425.02,639077.50
5,Australia and Oceania,Solomon Islands,Baby Food,Online,C,21/02/2015,159.42,759202.72,285087.64
6,Sub_Saharan Africa,Angola,Household,Offline,M,27/04/2011,502.54,2798046.49,693911.51
7,Sub_Saharan Africa,Burkina Faso,Vegetables,Online,H,27/07/2012,90.93,1245112.92,510216.66
8,Sub_Saharan Africa,Republic of the Congo,Personal Care,Offline,M,25/08/2015,56.67,496101.10,152114.20
9,Sub_Saharan Africa,Senegal,Cereal,Online,H,30/05/2014,117.11,1356180.10,584073.87


In [26]:
years = sales['Ship_Date'].str.split('/').str.get(2).values

sales['Ship_Date'] = years

In [27]:
sales["Country"].unique()

array(['Tuvalu', 'Grenada', 'Russia', 'Sao Tome and Principe', 'Rwanda',
       'Solomon Islands', 'Angola', 'Burkina Faso',
       'Republic of the Congo', 'Senegal', 'Kyrgyzstan', 'Cape Verde',
       'Bangladesh', 'Honduras', 'Mongolia', 'Bulgaria', 'Sri Lanka',
       'Cameroon', 'Turkmenistan', 'East Timor', 'Norway', 'Portugal',
       'New Zealand', 'Moldova ', 'France', 'Kiribati', 'Mali',
       'The Gambia', 'Switzerland', 'South Sudan', 'Australia', 'Myanmar',
       'Djibouti', 'Costa Rica', 'Syria', 'Brunei', 'Niger', 'Azerbaijan',
       'Slovakia', 'Comoros', 'Iceland', 'Macedonia', 'Mauritania',
       'Albania', 'Lesotho', 'Saudi Arabia', 'Sierra Leone',
       "Cote d'Ivoire", 'Fiji', 'Austria', 'United Kingdom', 'San Marino',
       'Libya', 'Haiti', 'Gabon', 'Belize', 'Lithuania', 'Madagascar',
       'Democratic Republic of the Congo', 'Pakistan', 'Mexico',
       'Federated States of Micronesia', 'Laos', 'Monaco', 'Samoa ',
       'Spain', 'Lebanon', 'Iran', 'Zamb

In [28]:
def convert_categorical_to_ordinal(df, columns):
  # Create a dictionary to map unique categories to ordinal values for each column
    ordinal_dicts = {}
    for col in columns:
        ordinal_dicts[col] = {}
        for i, row in df.iterrows():
            value = row[col]
            if value not in ordinal_dicts[col]:
                ordinal_dicts[col][value] = len(ordinal_dicts[col]) + 1
            df.at[i, col] = ordinal_dicts[col][value]

    return df

In [29]:
columns_to_convert = ["Region", "Country", "Sales_Channel", "Item_Type", "Order_Priority"]

for column in columns_to_convert:
    sales = convert_categorical_to_ordinal(sales.copy(), [column])  # Convert each column separately
    
sales

,Region,Country,Item_Type,Sales_Channel,Order_Priority,Ship_Date,Unit_Cost,Total_Revenue,Total_Profit
0,1,1,1,1,1,2010,159.42,2533654.00,951410.50
1,2,2,2,2,2,2012,117.11,576782.80,248406.36
2,3,3,3,1,3,2014,524.96,1158502.59,224598.75
3,4,4,4,2,2,2014,6.92,75591.66,19525.82
4,4,5,3,1,3,2013,524.96,3296425.02,639077.50
...,...,...,...,...,...,...,...,...,...
95,4,27,8,2,4,2011,35.84,97040.64,65214.72
96,5,75,4,1,3,2011,6.92,58471.11,15103.47
97,4,47,6,1,2,2016,90.93,228779.10,93748.05
98,7,61,7,1,4,2015,56.67,471336.91,144521.02


In [30]:
#expense
sales['Expense'] = sales['Total_Revenue'] -sales['Total_Profit']

#profit_margin
sales['Profit_Margin'] = (sales['Total_Profit'] / sales['Total_Revenue'] ) * 100

#rank
sales['Rank'] = sales['Profit_Margin'].rank(ascending=False).astype('int')

sales.sort_values(by='Profit_Margin', ascending=False).head(1)

,Region,Country,Item_Type,Sales_Channel,Order_Priority,Ship_Date,Unit_Cost,Total_Revenue,Total_Profit,Expense,Profit_Margin,Rank
67,2,56,8,1,4,2016,35.84,600821.44,403773.12,197048.32,67.203514,1


In [31]:
# from sklearn.preprocessing import OneHotEncoder

# categorical_cols = ["Region", "Country", "Item_Type","Sales_Channel","Order_Priority"]
# encoder = OneHotEncoder(handle_unknown='ignore')
# encoded_data = encoder.fit_transform(sales[categorical_cols])
# sales_encoded = pd.concat([sales, pd.DataFrame(encoded_data.toarray(), columns=encoder.get_feature_names_out())], axis=1)
# sales_encoded

## Creating a New Train and Validation Datasets

In [33]:
from sklearn.model_selection import train_test_split
data_train, data_val = train_test_split(sales, test_size = 0.2, random_state = 2)

## Classifying Predictors and Target

In [34]:
#Classifying Independent and Dependent Features

#Dependent Variable
Y_train = data_train.iloc[:, -1]
#Independent Variables
X_train = data_train.iloc[:,0 : -1]
#Independent Variables for Test Set
X_test = data_val.iloc[:,0 : -1]

actual_cost = list(data_val['Rank'])
actual_cost = np.asarray(actual_cost)

## Evaluating The Model With RMLSE

## Building the Lasso Regressor


In [54]:
#Lasso Regression
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_absolute_error,mean_squared_error

#Initializing the Lasso Regressor 
lasso_reg = Lasso()

#Fitting the Training data to the Lasso regressor
lasso_reg.fit(X_train,Y_train)

#Predicting for X_test
y_pred_lass =lasso_reg.predict(X_test)

#Printing the Score
print(f"""Lasso SCORE :
  MAE: {mean_absolute_error(y_pred_lass, actual_cost)}
  MSE: {mean_squared_error(y_pred_lass, actual_cost)}
  RMSE: {mean_squared_error(y_pred_lass, actual_cost, squared=False)}""")

Lasso SCORE :
  MAE: 6.598462169042703
  MSE: 73.43127227955654
  RMSE: 8.569204880241605
